In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from PIL import Image

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from pathlib import Path
import numpy as np

## Edit test_image_path variable in below cell to modify test images path

In [28]:
test_image_path='test-images'

In [29]:
##########################
# LOAD DATA
##########################
custom_transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),
                                       transforms.Resize((128, 128)),
                                       transforms.CenterCrop((120, 120)),
                                       transforms.ToTensor()])
root_path = Path(test_image_path)
test_dataset = ImageFolder(root=root_path, transform=custom_transform, is_valid_file=None)
print(test_dataset.class_to_idx)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=True, num_workers=0)

{'14-17': 0, '18-above': 1, '9-13': 2, 'below-9': 3}


In [30]:
##########################
# SETTINGS
##########################
# Architecture
NUM_CLASSES = 4
GRAYSCALE = False

In [31]:
##########################
# MODEL
##########################
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('something')
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes, grayscale):
        self.num_classes = num_classes
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1, padding=2)
        self.fc = nn.Linear(2048 * block.expansion, 1, bias=False)
        self.linear_1_bias = nn.Parameter(torch.zeros(self.num_classes-1).float())

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        logits = logits + self.linear_1_bias
        probas = torch.sigmoid(logits)
        return logits, probas


def resnet34(num_classes, grayscale):
    """Constructs a ResNet-34 model."""
    model = ResNet(block=BasicBlock,
                   layers=[3, 4, 6, 3],
                   num_classes=num_classes,
                   grayscale=grayscale)
    return model

something


In [32]:
#######################
### Initialize Model
#######################

model = resnet34(NUM_CLASSES, GRAYSCALE)
print('test')
model.load_state_dict(torch.load('model/model.pt', map_location=DEVICE))
model.eval()

test


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [33]:
#test loader class_to_idx:{'14-17': 0, '18-above': 1, '9-13': 2, 'below-9': 3}
#model class mapping directory_dic={'0':'below-9',
 #              '1':'9-13',
  #             '2':'14-17',
   #            '3':'18-above'}
predicted_label = []
true_label = []
for img, label in test_loader:
    img = img.to(DEVICE)
    true_label.append(label.item())
    with torch.set_grad_enabled(False):
        logits, probas = model(img)
        predict_levels = probas > 0.5
        pred_label = torch.sum(predict_levels, dim=1)
        predicted_age=pred_label.item()
        if predicted_age == 0:
            predicted_label.append(3)
        elif predicted_age == 1:
            predicted_label.append(2)
        elif predicted_age == 2:
            predicted_label.append(0)
        elif predicted_age == 3:
            predicted_label.append(1)
        #img.save('test-output'+directory_dict[3])
    
        

In [34]:
predicted_label = np.asarray(predicted_label)
true_label = np.asarray(true_label)

accuracy = accuracy_score(true_label, predicted_label)
precision = precision_score(true_label, predicted_label, average=None)
recall = recall_score(true_label, predicted_label, average=None)

In [35]:
#class_to_idx:{'14-17': 0, '18-above': 1, '9-13': 2, 'below-9': 3}
print('Accuracy: {:.3f}'.format(accuracy))
print('Precision')
print('14-17: {:.3f} 18-above: {:.3f} 9-13: {:.3f} below-9: {:.3f}'.format(precision[0], precision[1], precision[2], precision[3]))
print('Recall')
print('14-17: {:.3f} 18-above: {:.3f} 9-13: {:.3f} below-9: {:.3f}'.format(recall[0], recall[1], recall[2], recall[3]))

Accuracy: 0.512
Precision
14-17: 0.402 18-above: 0.840 9-13: 0.365 below-9: 0.373
Recall
14-17: 0.734 18-above: 0.382 9-13: 0.642 below-9: 0.339
